In [ ]:
### Load all the required libraries/modules
import os
import pandas as pd
import numpy as np

##tensorflow modules
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential

## text preprocessing modules
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import Word2Vec
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
### Load the data
review_data=pd.read_csv("/content/IMDB_Dataset.csv")

In [ ]:
print(review_data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [ ]:
review_data.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive


In [ ]:
### Take the reviews text as a list of elements and then perform the basic preprocessing
reviews= list(review_data['review'])
reviews[0:5]

["One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the f

In [ ]:
### Basic text preprocessing where we remove special characters, any tags etc
#### Removal of stopwords
text=[]
for r in reviews:
  text.append(remove_stopwords(r))
text[0:5]

["One reviewers mentioned watching 1 Oz episode you'll hooked. They right, exactly happened me.<br /><br />The thing struck Oz brutality unflinching scenes violence, set right word GO. Trust me, faint hearted timid. This pulls punches regards drugs, sex violence. Its hardcore, classic use word.<br /><br />It called OZ nickname given Oswald Maximum Security State Penitentary. It focuses mainly Emerald City, experimental section prison cells glass fronts face inwards, privacy high agenda. Em City home many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish more....so scuffles, death stares, dodgy dealings shady agreements far away.<br /><br />I main appeal fact goes shows wouldn't dare. Forget pretty pictures painted mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The episode I saw struck nasty surreal, I couldn't I ready it, I watched more, I developed taste Oz, got accustomed high levels graphic violence. Not violence, injustice (crooked guard

In [ ]:
### Removal of tags/special characters

### Basic text preprocessing where we remove special characters, any tags etc
#### Removal of stopwords
#### Removal of tags
text=[]
for r in reviews:
  r = re.sub(r'<.*?>',"",r)
  text.append(remove_stopwords(r))
text[0:5]

["One reviewers mentioned watching 1 Oz episode you'll hooked. They right, exactly happened me.The thing struck Oz brutality unflinching scenes violence, set right word GO. Trust me, faint hearted timid. This pulls punches regards drugs, sex violence. Its hardcore, classic use word.It called OZ nickname given Oswald Maximum Security State Penitentary. It focuses mainly Emerald City, experimental section prison cells glass fronts face inwards, privacy high agenda. Em City home many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish more....so scuffles, death stares, dodgy dealings shady agreements far away.I main appeal fact goes shows wouldn't dare. Forget pretty pictures painted mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The episode I saw struck nasty surreal, I couldn't I ready it, I watched more, I developed taste Oz, got accustomed high levels graphic violence. Not violence, injustice (crooked guards who'll sold nickel, inmates who'll

In [ ]:
### Basic text preprocessing where we remove special characters, any tags etc
#### Removal of stopwords
#### Removal of tags
#### Removal of special characters
#### Removal of email id
#### Removal of numbers
text=[]
for r in reviews:
  r = re.sub(r'<.*?>',"",r)
  r = re.sub(r'\S+@\S+\s','',r)
  r = re.sub(r'\d+','',r)
  r = re.sub(r'[#$!\*\)\(\\%:;,\'_-]','',r)
  text.append(remove_stopwords(r))
text[0:5]

['One reviewers mentioned watching Oz episode youll hooked. They right exactly happened me.The thing struck Oz brutality unflinching scenes violence set right word GO. Trust faint hearted timid. This pulls punches regards drugs sex violence. Its hardcore classic use word.It called OZ nickname given Oswald Maximum Security State Penitentary. It focuses mainly Emerald City experimental section prison cells glass fronts face inwards privacy high agenda. Em City home many..Aryans Muslims gangstas Latinos Christians Italians Irish more....so scuffles death stares dodgy dealings shady agreements far away.I main appeal fact goes shows wouldnt dare. Forget pretty pictures painted mainstream audiences forget charm forget romance...OZ doesnt mess around. The episode I saw struck nasty surreal I I ready I watched I developed taste Oz got accustomed high levels graphic violence. Not violence injustice crooked guards wholl sold nickel inmates wholl kill order away mannered middle class inmates turn

In [ ]:
### Now the reviews are faily clean, we will tokenize the reviews and get word embeddings for these
tokenizer=Tokenizer(num_words=5000,lower=True,oov_token='UNK')

In [ ]:
## Before applying the tokenizer, lets split the data into train test
train_text=text[:len(text)-500]
test_text=text[-500:]
print(len(text),len(train_text),len(test_text))

50000 49500 500


In [ ]:
### Now lets tokenize the reviews
tokenizer.fit_on_texts(train_text)

In [ ]:
tokenizer.word_index

{'UNK': 1,
 'i': 2,
 'the': 3,
 'movie': 4,
 'film': 5,
 'it': 6,
 'like': 7,
 'this': 8,
 'good': 9,
 'time': 10,
 'story': 11,
 'bad': 12,
 'great': 13,
 'people': 14,
 'dont': 15,
 'movies': 16,
 'and': 17,
 'films': 18,
 'way': 19,
 'but': 20,
 'characters': 21,
 'think': 22,
 'watch': 23,
 'its': 24,
 'in': 25,
 'a': 26,
 'seen': 27,
 'character': 28,
 'love': 29,
 'acting': 30,
 'plot': 31,
 'best': 32,
 'know': 33,
 'little': 34,
 'life': 35,
 'there': 36,
 'better': 37,
 'if': 38,
 'end': 39,
 'scene': 40,
 'man': 41,
 'scenes': 42,
 's': 43,
 'he': 44,
 'im': 45,
 'watching': 46,
 'thing': 47,
 'real': 48,
 'actors': 49,
 'doesnt': 50,
 'years': 51,
 'didnt': 52,
 'funny': 53,
 'actually': 54,
 'makes': 55,
 'work': 56,
 'look': 57,
 'going': 58,
 'director': 59,
 'lot': 60,
 'new': 61,
 'one': 62,
 'you': 63,
 'old': 64,
 'thats': 65,
 'things': 66,
 'cast': 67,
 'want': 68,
 'pretty': 69,
 'all': 70,
 'world': 71,
 'horror': 72,
 'young': 73,
 'got': 74,
 'fact': 75,
 'what'

In [ ]:
len(tokenizer.word_index)

148958

In [ ]:
#### Now we need embeddings for these words
## Get the glove vectors
embeddings_index= dict()
glove= open('/content/glove.6B.100d.txt','r',encoding='utf-8')
for line in glove:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
glove.close()

In [ ]:
len(embeddings_index)

252338

In [ ]:
### Create an embedding matrix for the vocabulary created for the reviews
vocab= len(tokenizer.word_index)+1
embedding_matrix = np.zeros((vocab, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(148959, 100)

In [ ]:
embedding_matrix[90]

array([-0.53086001,  0.51403999,  0.087599  , -0.37314001,  0.27469999,
        0.07947   , -0.0085023 ,  0.028399  , -0.35113999,  0.094339  ,
        0.087771  , -0.38306999,  0.43129   ,  0.15261   , -0.1512    ,
       -0.46070001,  0.080433  ,  0.037627  , -0.43959001,  0.42451   ,
        0.16057999,  0.26607999,  0.35310999,  0.014055  , -0.052771  ,
       -0.16150001, -0.29899999, -0.56213999, -0.18742   ,  0.044237  ,
       -0.28117999,  0.36594   , -0.26225999,  0.11013   ,  0.44358   ,
        0.43131   , -0.0053095 ,  0.34705001, -0.44883001, -0.33726999,
       -0.13281   , -0.35541999, -0.081663  , -0.12983   ,  0.080606  ,
       -0.161     ,  0.36700001, -0.30568001,  0.057269  , -0.79400003,
       -0.24581   ,  0.027115  ,  0.13203   ,  1.22619998, -0.19182999,
       -2.54970002,  0.055273  , -0.13779999,  1.45519996,  0.53697002,
       -0.12337   ,  1.12779999, -0.16365001,  0.21871001,  0.82735002,
       -0.30680999,  0.65456003,  0.17636   ,  0.61720002, -0.18

In [ ]:
train_indices=tokenizer.texts_to_sequences(train_text)

In [ ]:
train_indices[0:3]

[[62,
  1842,
  916,
  46,
  2936,
  255,
  329,
  3035,
  88,
  84,
  458,
  435,
  202,
  3,
  47,
  3155,
  2936,
  1,
  1,
  42,
  429,
  150,
  84,
  523,
  877,
  1585,
  1,
  1,
  1,
  8,
  2406,
  1,
  1,
  1300,
  248,
  429,
  24,
  3253,
  216,
  213,
  523,
  6,
  336,
  2936,
  1,
  212,
  1,
  1,
  2455,
  915,
  1,
  6,
  2579,
  1247,
  1,
  388,
  4469,
  2363,
  1046,
  1,
  2821,
  1,
  262,
  1,
  1,
  185,
  4831,
  3481,
  388,
  206,
  988,
  1,
  1,
  1,
  1,
  4988,
  1,
  2330,
  501,
  83,
  1,
  188,
  1,
  1,
  1,
  1,
  1,
  106,
  113,
  2,
  144,
  1164,
  75,
  133,
  126,
  416,
  2888,
  672,
  69,
  1118,
  4223,
  2384,
  946,
  672,
  1268,
  672,
  728,
  2936,
  50,
  784,
  1548,
  3,
  255,
  2,
  93,
  3155,
  1456,
  2089,
  2,
  2,
  1433,
  2,
  149,
  2,
  1347,
  1111,
  2936,
  74,
  1,
  185,
  1886,
  1999,
  429,
  82,
  429,
  1,
  1,
  4697,
  1,
  2866,
  1,
  1,
  1,
  357,
  480,
  113,
  1,
  609,
  673,
  1,
  516,
  1046,
  1,

In [ ]:
## As mentioned we are limiting the number of words say 64
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length=64
train_indices=pad_sequences(train_indices,maxlen=max_length,padding='post')

In [ ]:
### Creating the model architecture
model= Sequential()
model.add(Embedding(input_dim=vocab,output_dim=100,input_length=max_length,weights=[embedding_matrix],trainable=False))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │      14,895,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,895,900 (56.82 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,895,900 (56.82 MB)

None


In [ ]:
review_data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
review_data['sentiment'].replace(['positive','negative'],[1,0],inplace=True)
review_data['sentiment'].value_counts()

<ipython-input-24-528c840d9186>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  review_data['sentiment'].replace(['positive','negative'],[1,0],inplace=True)
<ipython-input-24-528c840d9186>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  review_data['sentiment'].replace(['positive','negative'],[1,0

,count
sentiment,
1,25000
0,25000


In [ ]:
Y=list(review_data['sentiment'])
train_y=Y[:len(Y)-500]
test_y=Y[-500:]
print(len(train_y),len(test_y))

49500 500


In [ ]:
train_indices=np.asarray(train_indices)
train_y=np.asarray(train_y)

In [ ]:
train_indices[1]

array([ 410, 2149,  361, 4406, 2385,   74,    1, 2102, 3137,   63,  222,
          1,  636,    1, 1652, 1556,    1,    1,  140,   46,    1,  293,
       2225,  270,   26, 4184,  221,   13, 3083,   86,   35,    3, 1724,
        134,  206,   34,   66,  884, 2871,  213, 2040,  841, 2971, 1108,
       1011, 4779,    6,  158, 1681, 4313,  436,   42, 3482,    1,    1,
        561,  436,  935,    1,    1,    1, 2132, 1796, 1153], dtype=int32)

In [ ]:
model.fit(train_indices,train_y,batch_size=32,epochs=10)

Epoch 1/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - accuracy: 0.7410 - loss: 0.5185
Epoch 2/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8240 - loss: 0.3948
Epoch 3/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.8421 - loss: 0.3556
Epoch 4/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8512 - loss: 0.3386
Epoch 5/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.8576 - loss: 0.3239
Epoch 6/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - accuracy: 0.8666 - loss: 0.3080
Epoch 7/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8724 - loss: 0.2951
Epoch 8/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8786 - loss: 0.2848
Epoch 9/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.8804 - loss: 0.2785
Epoch 10/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.8880 - loss: 0.2658


In [ ]:
test_indices=tokenizer.texts_to_sequences(test_text)
test_indices=pad_sequences(test_indices,maxlen=max_length,padding='post')

test_indices=np.asarray(test_indices)
test_y=np.asarray(test_y)

In [ ]:
# Final evaluation of the model
scores = model.evaluate(test_indices, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 83.20%


In [ ]:
scores

[0.4029068648815155, 0.8320000171661377]

In [ ]:
## Creating the model architecture
model= Sequential()
model.add(Embedding(input_dim=vocab,output_dim=100,input_length=max_length,weights=[embedding_matrix],trainable=False))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │      14,895,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,895,900 (56.82 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,895,900 (56.82 MB)

None


In [ ]:
model.fit(train_indices,train_y,batch_size=32,epochs=5)

Epoch 1/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.5817 - loss: 0.6703
Epoch 2/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 19s 6ms/step - accuracy: 0.6343 - loss: 0.6363
Epoch 3/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6610 - loss: 0.6245
Epoch 4/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.5891 - loss: 0.6662
Epoch 5/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.5997 - loss: 0.6591


In [ ]:
# Final evaluation of the model
scores = model.evaluate(test_indices, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 64.20%


In [ ]:
scores

[0.6290221214294434, 0.6420000195503235]

In [ ]:
### Will the batch normalization work
## Creating the model architecture
model= Sequential()
model.add(Embedding(input_dim=vocab,output_dim=100,input_length=max_length,weights=[embedding_matrix],trainable=False))
model.add(SimpleRNN(32))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │      14,895,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,895,900 (56.82 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,895,900 (56.82 MB)

None


In [ ]:
model.fit(train_indices,train_y,batch_size=32,epochs=10)
# Final evaluation of the model
scores = model.evaluate(test_indices, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 13s 7ms/step - accuracy: 0.5776 - loss: 0.7001
Epoch 2/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 18s 6ms/step - accuracy: 0.5599 - loss: 0.6774
Epoch 3/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.5738 - loss: 0.6680
Epoch 4/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.5849 - loss: 0.6630
Epoch 5/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.5987 - loss: 0.6550
Epoch 6/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6334 - loss: 0.6349
Epoch 7/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6064 - loss: 0.6485
Epoch 8/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6175 - loss: 0.6437
Epoch 9/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.6370 - loss: 0.6286
Epoch 10/10
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.6129 - loss: 0.6479
Accuracy: 59.60%


In [ ]:
### Execute the same with Keras Embedding
model= Sequential()
model.add(Embedding(input_dim=vocab,output_dim=100,input_length=max_length))
model.add(SimpleRNN(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_2 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
model.fit(train_indices,train_y,batch_size=32,epochs=3)
# Final evaluation of the model
scores = model.evaluate(test_indices, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/3
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.6746 - loss: 0.5859
Epoch 2/3
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 19s 9ms/step - accuracy: 0.8572 - loss: 0.3441
Epoch 3/3
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - accuracy: 0.8853 - loss: 0.2833
Accuracy: 82.20%


In [ ]:
### Execute the same with Keras Embedding
model= Sequential()
model.add(Embedding(input_dim=vocab,output_dim=100,input_length=max_length))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
model.fit(train_indices,train_y,batch_size=32,epochs=5)
# Final evaluation of the model
scores = model.evaluate(test_indices, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 24s 14ms/step - accuracy: 0.7708 - loss: 0.4560
Epoch 2/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - accuracy: 0.8837 - loss: 0.2802
Epoch 3/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 42s 14ms/step - accuracy: 0.9086 - loss: 0.2252
Epoch 4/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 40s 14ms/step - accuracy: 0.9300 - loss: 0.1779
Epoch 5/5
1547/1547 ━━━━━━━━━━━━━━━━━━━━ 41s 14ms/step - accuracy: 0.9457 - loss: 0.1434
Accuracy: 84.60%
